In [1]:
import onnx
import numpy as np
import onnxruntime as ort

2022-10-19 15:40:47.988708 [E:onnxruntime:Default, provider_bridge_ort.cc:902 Ensure] Failed to load library libonnxruntime_providers_shared.dylib with error: dlopen(libonnxruntime_providers_shared.dylib, 0x000A): tried: '/opt/anaconda3/envs/d2l/lib/python3.8/site-packages/onnxruntime/capi/libonnxruntime_providers_shared.dylib' (no such file), '/opt/anaconda3/envs/d2l/lib/libonnxruntime_providers_shared.dylib' (no such file), '/opt/anaconda3/envs/d2l/bin/../lib/libonnxruntime_providers_shared.dylib' (no such file), 'libonnxruntime_providers_shared.dylib' (no such file), '/usr/local/lib/libonnxruntime_providers_shared.dylib' (no such file), '/usr/lib/libonnxruntime_providers_shared.dylib' (no such file), '/Users/bob/libonnxruntime_providers_shared.dylib' (no such file)
2022-10-19 15:40:47.988730 [W:onnxruntime:Default, onnxruntime_pybind_state.cc:1615 pybind11_init_onnxruntime_pybind11_state] Init provider bridge failed.


In [2]:
onnx_model_path = "/Users/bob/code/CodeReading/Paddle-Lite/pnn/3x3s2_dw.onnx"

In [3]:
data = np.full((1,32,104,104), 1, dtype=np.float32)

In [4]:
sess = ort.InferenceSession(onnx_model_path)

In [5]:
inputs = {"data" : data}

In [6]:
outputs = sess.run(None, inputs)

In [7]:
outputs[0].flatten()[0:100]

array([-0.12178007, -0.12580684, -0.12580684, -0.12580684, -0.12580684,
       -0.12580684, -0.12580684, -0.12580684, -0.12580684, -0.12580684,
       -0.12580684, -0.12580684, -0.12580684, -0.12580684, -0.12580684,
       -0.12580684, -0.12580684, -0.12580684, -0.12580684, -0.12580684,
       -0.12580684, -0.12580684, -0.12580684, -0.12580684, -0.12580684,
       -0.12580684, -0.12580684, -0.12580684, -0.12580684, -0.12580684,
       -0.12580684, -0.12580684, -0.12580684, -0.12580684, -0.12580684,
       -0.12580684, -0.12580684, -0.12580684, -0.12580684, -0.12580684,
       -0.12580684, -0.12580684, -0.12580684, -0.12580684, -0.12580684,
       -0.12580684, -0.12580684, -0.12580684, -0.12580684, -0.12580684,
       -0.12580684, -0.12580684,  0.09853861,  0.23208767,  0.23208767,
        0.23208767,  0.23208767,  0.23208767,  0.23208767,  0.23208767,
        0.23208767,  0.23208767,  0.23208767,  0.23208767,  0.23208767,
        0.23208767,  0.23208767,  0.23208767,  0.23208767,  0.23

## Construct ConvDepthwise Model

In [8]:
n=1
h=9
w=9
ic=2
oc=2
kh=3
kw=3
sy=2
sx=2
dy=2
dx=1
oh=3
ow=4
fOpFlag=0
groups = 2

In [9]:
import torch
from functools import reduce # Valid in Python 2.6+, required in Python 3
from torch import nn as nn
import operator
import numpy as np

/opt/anaconda3/envs/d2l/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
 m = nn.Conv2d(ic, oc , (kh,kw), (sy,sx),dilation = (dy,dx),padding = 0, groups = ic)

In [11]:
def get_data(shape):
    size = reduce(operator.mul, shape, 1)
    data = []
    for i in range(size):
        data.append(i % 5)
    
    res = np.array(data).astype(np.float32).reshape(shape)
    return res

In [12]:
# data = torch.from_numpy(get_data((n, ic, h, w)))
raw = np.transpose(get_data((n, h, w, ic)), (0,3,1,2))

In [13]:
data = torch.from_numpy(raw)

In [14]:
def gen_pnn_weight(shape):
    size = reduce(operator.mul, shape, 1)
    data = []
    for i in range(size):
        data.append(i % 4)
    
    return np.array(data).astype(np.float32).reshape(shape)

In [15]:
def gen_pnn_bias(shape):
    size = reduce(operator.mul, shape, 1)
    data = []
    for i in range(size):
        data.append(i % 3)
    
    return np.array(data).astype(np.float32).reshape(shape)

In [16]:
pnn_weight = gen_pnn_weight((1,kh,kw,oc))

In [17]:
# 这里跟转换工具保持一致
torch_weitht = np.transpose(pnn_weight, (0,3,1,2)).reshape((oc, ic // groups, kh, kw ))

In [18]:
pnn_bias = gen_pnn_bias((oc,))

### change weight

In [19]:
pnn_weight.shape

(1, 3, 3, 2)

In [20]:
pnn_bias

array([0., 1.], dtype=float32)

In [21]:
m.weight = torch.nn.Parameter(torch.from_numpy(torch_weitht))
# m.weight = torch.nn.Parameter(torch.from_numpy(pnn_weight.reshape((32,1,3,3))))

In [22]:
m.bias = torch.nn.Parameter(torch.from_numpy(pnn_bias))

### Calc output

In [23]:
pnn_bias

array([0., 1.], dtype=float32)

In [24]:
output = m(data)

In [25]:
# silu
a = nn.SiLU()
final = a(output)

In [26]:
# transpose
ooo = torch.permute(final,(0,2,3,1))

In [27]:
ooo.flatten()[21]

tensor(29., grad_fn=<SelectBackward0>)

In [28]:
sample_input = data[0,31,86:89,100:103]

IndexError: index 31 is out of bounds for dimension 1 with size 2

In [ ]:
sample_input

In [ ]:
sample_kernel = torch_weitht[31,0]

In [ ]:
sample_kernel

In [ ]:
sample_bias = pnn_bias[31]

In [ ]:
sample_bias

In [ ]:
(sample_input * sample_kernel).sum() + sample_bias

In [ ]:
output[0,0,50,0:4]

In [ ]:
output.flatten()[1600:1604]

In [ ]:
torch.onnx.export(m,               # model being run
                  data,                         # model input (or a tuple for multiple inputs)
                  "/Users/bob/Desktop/tmp/conv_opt/conv3x3_s2p0.onnx",   # where to save the model (can be a file or file-like object)
               export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=10,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['data'],   # the model's input names
                  output_names = ['output'], # the model's output names
                 )

In [ ]:
np.unravel_index(21, (1,9,5,3))

In [ ]:
output.flatten()

In [ ]:
output[0,2,2,1]

In [ ]:
output.shape

In [ ]:
ms = nn.SiLU()

In [ ]:
ms(output).flatten()[21]

### Generate Conv2D 3x3

In [29]:
s13x3 = nn.Conv2d(3, 12 , (3,3), (1,1),dilation = (1,1),padding = 0)

In [31]:
n = 1
h = 114
w = 114
ic = 3

In [32]:
data = torch.from_numpy(get_data((n, ic, h, w)))

In [36]:
torch.onnx.export(s13x3,               # model being run
                  data,                         # model input (or a tuple for multiple inputs)
                  "/Users/bob/Desktop/tmp/conv_opt/conv3x3_s1p0.onnx",   # where to save the model (can be a file or file-like object)
                   export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=10,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['data'],   # the model's input names
                  output_names = ['output'], # the model's output names
                 )